In [1339]:
import numpy as np

In [1340]:
class ANN:
    w_l_ij= None
    b_l_j = None
    a_l_ij = None
    delta_l_ij = None
    input_featureCount = None
    input_count = None
    layer_info = None
    dl_dw = None
    diff = {}
    def __init__(self,xShape,rseed = 10,listLayer= None):
        np.random.seed(rseed)
        inputCount,inputDim = xShape
        self.input_featureCount = inputDim
        self.input_count = inputCount
        self.layer_info = [['None','None',self.input_featureCount]]
        self.w_l_ij = ['None']
        self.b_l_j = ['None']
        self.optimizer = None
        self.diff[ANN.activation_linear] = ANN.diff_activation_linear
        self.diff[ANN.activation_sigmoid] = ANN.diff_activation_sigmoid
        self.diff[ANN.activation_tanh] = ANN.diff_activation_tanh

    def compile(self,optimizer,lossFunction):
        self.optimizer = optimizer
        self.layer_info.append(['Loss',lossFunction])

    def __str__(self):
        return

    def addLayers_Dense(self,neuronCount,activationFunction):
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.layer_info[-2][2],neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)

    def addLayers_softmax(self):
        self.layer_info.append(['Softmax',self.layer_info[-1][2]])


    def forward_layer_dense(self,layerNo):
        if layerNo < 1:
            print("ERRRROR")
        output = self.layer_info[layerNo][1](self.a_l_ij[layerNo-1] @ self.w_l_ij[layerNo] + self.b_l_j[layerNo]) 
        self.a_l_ij.append(output)
    
    def forward_layer_softmax(self,layerNo):
        if layerNo < 1: 
            print("ERRORRR")
        denom = np.sum( np.exp(self.a_l_ij[layerNo-1]) )
        output = np.exp(self.a_l_ij[layerNo]) / denom
        self.a_l_ij.append(output)

    #Create cross entropy loss 
    def forward_layer_loss(self,layerNo,y):
        y_hat  =  self.a_l_ij[-1]
        lossFunction = self.layer_info[layerNo][1]
        return lossFunction(y_hat,y)

    def forwardProp(self,x,y):
        self.a_l_ij = []
        self.a_l_ij.append(x)
        for layerNo in range(len(self.layer_info)):
            if self.layer_info[layerNo][0] == 'Dense':
                self.forward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Softmax':
                self.forward_layer_softmax(layerNo)
            if self.layer_info[layerNo][0] == 'Loss':
                cost = self.forward_layer_loss(layerNo,y)
                return cost
            
    #Handle softmax backpropogation
    def backwardProp(self,y):
        self.dl_dw = []
        lastIndex = len(self.layer_info) - 2
        self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        self.delta_l_ij[lastIndex] = (2 * (self.a_l_ij[lastIndex] - y).T * self.diff[self.layer_info[lastIndex][1]](self.a_l_ij[lastIndex]))
        self.dl_dw.append(self.a_l_ij[lastIndex-1].T @ self.delta_l_ij[lastIndex])
        for layerNo in range(lastIndex-1,0,-1):
            self.delta_l_ij[layerNo] = (self.w_l_ij[layerNo+1] @self.delta_l_ij[layerNo+1].T).T * self.diff[self.layer_info[layerNo][1]](self.a_l_ij[layerNo])
            self.dl_dw.append(self.a_l_ij[layerNo-1].T @ self.delta_l_ij[layerNo])
        self.dl_dw.append(['None'])
        self.dl_dw.reverse()

    def fit(self,x,y):
        #Hyperparameters if required: 
        epoch = 100
        eta = 1e-7
        self.optimizer(self,x,y,eta,epoch)

    @staticmethod    
    def optimizer_gradientDescent(Obj,x,y,eta,epoch):
        for i in  range(epoch):
            errorSum = 0
            print("Epoch:",i,end="\t")
            for j in range(y.shape[1]):
                #x and y both are 2d matrix
                x_1xd = x[j:j+1]
                y_1xk = y[j:j+1]
                errorSum += Obj.forwardProp(x_1xd,y_1xk)
                Obj.backwardProp(y_1xk)
                for index in range(1,len(Obj.w_l_ij)):
                    Obj.w_l_ij[index] = Obj.w_l_ij[index] - (eta * Obj.dl_dw[index])
                    Obj.b_l_j[index] = Obj.b_l_j[index] - (eta * Obj.delta_l_ij[index])
            print("Error:",errorSum)

    @staticmethod
    def loss_MSE(P,y):
        return np.sum((P - y)**2)   
      
    @staticmethod
    def activation_linear(z):
        return z
    
    @staticmethod
    def diff_activation_linear(x):
        return np.ones(x.shape)
    
    @staticmethod
    def activation_sigmoid(z):    
        return 1 / (1 + np.exp(-z))
    
    @staticmethod    
    def diff_activation_sigmoid(a):
        return a * (1- a)

    @staticmethod
    def activation_tanh(z):
        sinh = np.exp(z) - np.exp(-z)
        cosh = np.exp(z) + np.exp(-z)
        return sinh / cosh
    
    @staticmethod
    def diff_activation_tanh(z):
        return 1 - (z * z)

    @staticmethod
    def activation_relu(z):
        pass

In [1341]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [1342]:
dictt = sk.fetch_california_housing()
x = dictt.data
y = np.array([dictt.target])
y = y.T
x[0:1].shape

(1, 8)

In [1343]:
model = ANN(x.shape,20)

In [1344]:
model.addLayers_Dense(4,ANN.activation_linear)
model.addLayers_Dense(6,ANN.activation_linear)
model.addLayers_Dense(1,ANN.activation_linear)

In [1345]:
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)

In [1346]:
model.layer_info

[['None', 'None', 8],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 4],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 6],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 1],
 ['Loss', <function __main__.ANN.loss_MSE(P, y)>]]

In [1347]:
model.dl_dw

In [1348]:
import copy
w_old = copy.deepcopy(model.w_l_ij)

In [1349]:
model.fit(x,y)

Epoch: 0	Error: 251709.97881602013
Epoch: 1	Error: 125268.39841705753
Epoch: 2	Error: 68850.51753695696
Epoch: 3	Error: 39733.013342268096
Epoch: 4	Error: 23556.165663753443
Epoch: 5	Error: 14188.581480619716
Epoch: 6	Error: 8629.192623727628
Epoch: 7	Error: 5279.915120578846
Epoch: 8	Error: 3243.08957501485
Epoch: 9	Error: 1996.9995332569224
Epoch: 10	Error: 1231.724113551302
Epoch: 11	Error: 760.5511326318039
Epoch: 12	Error: 469.9690146659486
Epoch: 13	Error: 290.55935191546155
Epoch: 14	Error: 179.70409894915113
Epoch: 15	Error: 111.17139862458333
Epoch: 16	Error: 68.78739630046312
Epoch: 17	Error: 42.56803713323724
Epoch: 18	Error: 26.345227138079196
Epoch: 19	Error: 16.30620168480729
Epoch: 20	Error: 10.093184182364672
Epoch: 21	Error: 6.247729512009778
Epoch: 22	Error: 3.8675014698521166
Epoch: 23	Error: 2.3941409297045735
Epoch: 24	Error: 1.4820996250169503
Epoch: 25	Error: 0.9175117697565603
Epoch: 26	Error: 0.5680034744225421
Epoch: 27	Error: 0.3516367982788072
Epoch: 28	Erro

In [1350]:
w_new = model.w_l_ij

In [1351]:
model.delta_l_ij

[0,
 array([[4.78801367e-08, 3.17254394e-08, 7.19776715e-08, 5.57076439e-08]]),
 array([[1.28915904e-08, 9.45604812e-09, 1.98825850e-08, 8.92865054e-10,
         2.31556058e-08, 3.35285234e-08]]),
 array([[3.77860037e-08]])]

In [1352]:
model.a_l_ij

[array([[   8.3252    ,   41.        ,    6.98412698,    1.02380952,
          322.        ,    2.55555556,   37.88      , -122.23      ]]),
 array([[-39.81888021, 192.52351321, -50.99432637,  -8.04885399]]),
 array([[ -5.74808185,  38.41390014, -12.01629666,  73.40193449,
         -46.63852523,  32.85372759]]),
 array([[4.52600002]])]

In [1353]:
model.b_l_j

['None',
 array([[0.26826643, 0.066933  , 0.77065132, 0.48035725]]),
 array([[0.50111345, 0.03697463, 0.70789835, 0.62040898, 0.77755873,
         0.4590402 ]]),
 array([[0.89083326]])]

In [1354]:
w_old,w_new

(['None',
  array([[0.5881308 , 0.89771373, 0.89153073, 0.81583748],
         [0.03588959, 0.69175758, 0.37868094, 0.51851095],
         [0.65795147, 0.19385022, 0.2723164 , 0.71860593],
         [0.78300361, 0.85032764, 0.77524489, 0.03666431],
         [0.11669374, 0.7512807 , 0.23921822, 0.25480601],
         [0.85762553, 0.94977903, 0.56168686, 0.17878052],
         [0.77025193, 0.49238104, 0.63125307, 0.83949792],
         [0.4610394 , 0.49794007, 0.67941112, 0.65078591]]),
  array([[0.32920641, 0.51064106, 0.26362883, 0.31051155, 0.62685344,
          0.55744981],
         [0.31857956, 0.39484322, 0.25797459, 0.58224112, 0.16162871,
          0.59813382],
         [0.82582358, 0.15639172, 0.73430052, 0.40864343, 0.7786879 ,
          0.80397057],
         [0.78607144, 0.59228702, 0.6644892 , 0.64656729, 0.42563648,
          0.51356833]]),
  array([[0.37980555],
         [0.2918922 ],
         [0.55722886],
         [0.0841636 ],
         [0.63128167],
         [0.94457049]])],
 